In [1]:
import sympy as sp
from sympy import *
from sympy.abc import *
from sympy.parsing.sympy_parser import parse_expr

In [2]:
def given(fx):
    li = [
        "1+-2.3+4+5.-6",
        "1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15",
        "1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)",
        "(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))",
        "1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))" 
    ]
    li_ret = []
    for cl in li:
        try:
            li_ret.append(fx(cl))
        except Exception as e:
            print(e)
            li_ret.append('{e.message}')
    return li_ret
        

In [3]:
clause = '1+-2.3+4+5.6'

In [4]:
def te(cl):
    print(cl)
    return cl

In [5]:
given(te)

1+-2.3+4+5.-6
1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15
1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)
(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))
1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))


['1+-2.3+4+5.-6',
 '1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15',
 '1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)',
 '(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))',
 '1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))']

In [6]:
max_number = float('inf')

In [7]:
def max_no(clause):
    final = []
    clause = clause.replace('(', '')
    clause = clause.replace(')', '')
    dot_split = clause.split('.')
    for ds in dot_split:
        final.extend(ds.split('+'))
    for i in range(len(final)):
        final[i] = abs(int(final[i]))
    return max(final)



In [8]:
def define_variables(max_no):
    return symbols(f'x:{max_no}')

In [9]:
def replace_nums_with_symbols(clause):
    max_num = max_no(clause)
    clause = clause.replace('-', '~')
    split_by_and = clause.split('.')
    syms = define_variables(max_num+1)
    for i in range(max_num+1):
        exec(f'x{i} = syms[{i}]')
    fin_clause = ''
    for sub_clause in split_by_and:
        fin_sub_clause = ''
        for sub_sub_clause in sub_clause.split('+'):
            for i in range(max_num, 0, -1):
                if sub_sub_clause.replace(str(i), str(syms[i])) != sub_sub_clause:
                    sub_sub_clause = sub_sub_clause.replace(str(i), str(syms[i]))
                    break
            fin_sub_clause = fin_sub_clause + '|' + sub_sub_clause
        
        fin_clause = fin_clause + '&' + fin_sub_clause[1:]
    return fin_clause[1:]

In [10]:
li = given(replace_nums_with_symbols)

In [11]:
def auto_convert_to_cnf(clause):
    
    converted_clause_string = replace_nums_with_symbols(clause)
    try:
        exec(f'converted_clause = {converted_clause_string}')
        print(clause, sp.to_cnf(converted_clause_string))
        print()
    except:
        print(f'Error in clause {clause}')
        print()

In [12]:
def get_sp_clause(clause):
    converted_clause_string = replace_nums_with_symbols(clause)
    try:
        return parse_expr(converted_clause_string)
    except Exception as e:
        print(f'Error in clause {clause} {e}')
        print()

In [13]:
given(get_sp_clause)

Error in clause 1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1) ('EOF in multi-line statement', (2, 0))



[x1 | x4 | (x3 & ~x2) | (x5 & ~x6),
 x12 | x15 | (x4 & x6) | (x1 & (x2 | (x3 & x5) | (x3 & ~x1))),
 None,
 x12 & ~x1 & (x15 | x16) & (x17 | x18) & (x4 | x6) & (x1 | (x2 & (x3 | x5) & (x3 | ~x1))),
 x1 & x2 & x3 & x4 & ~x1 & ~x2 & ~x5]

In [14]:
def subformulas(expr):
# subformula of expr without atoms
# considering operator as binary
    
    if expr.func == sp.Symbol:
        return []
    elif len(expr.args)==1:
        return [expr] + subformulas(expr.args[0])
    elif len(expr.args)==2:
        return [expr] + subformulas(expr.args[0]) + subformulas(expr.args[1]) 
    elif len(expr.args)>2:
        return [expr] + subformulas(expr.args[0]) + subformulas(eval(str(expr.func)+str(expr.args[1:])))  
    else:
        return "error"

In [31]:
syms = define_variables(18+1)
for i in range(18+1):
    exec(f'x{i} = syms[{i}]')

In [20]:
subformulas((x1 | x4 | (x3 & ~x2) | (x5 & ~x6)))

[x1 | x4 | (x3 & ~x2) | (x5 & ~x6),
 x4 | (x3 & ~x2) | (x5 & ~x6),
 (x3 & ~x2) | (x5 & ~x6),
 x3 & ~x2,
 ~x2,
 x5 & ~x6,
 ~x6]

In [21]:
def depth(expr):
# still considering operator as binary
    if expr.func == Not:
        return 1 + depth(expr.args[0])
    elif (len(expr.args)==2):
        return 1 + max([depth(x) for x in expr.args])
    elif ( len(expr.args)>2):
        #return 1 + max(depth(expr.args[0]), depth(eval(str(expr.func)+str(expr.args[1:]))))
        return len(expr.args)-1 + max([depth(x) for x in expr.args])
    elif expr.func == Symbol:
        return 0
    else: 
        return"error"

In [22]:
depth(x1 | x4 | (x3 & ~x2) | (x5 & ~x6))

5

In [23]:
F = x1 | x4 | (x3 & ~x2) | (x5 & ~x6)
sorted(subformulas(F), key=depth)

[~x2,
 ~x6,
 x3 & ~x2,
 x5 & ~x6,
 (x3 & ~x2) | (x5 & ~x6),
 x4 | (x3 & ~x2) | (x5 & ~x6),
 x1 | x4 | (x3 & ~x2) | (x5 & ~x6)]

In [24]:
def symbol_init(x, count=0):
    def new_symbol ():
        nonlocal count
        count += 1
        return var(str(x) + str(count)) #var add the new symbol in the name space
    return new_symbol

In [25]:
def binSubs(x, y, z):
    #replace y by z in x
    #print("subs", x, y, z)
    if (len(y.args) >0) and (len(x.args)>2) and (y!=x) and (y.func==x.func):
     
        if all([y in x.args for y in y.args]): 
            l = list(x.args)
            for i in y.args:
                l.remove(i)
            if (z.func== Symbol):
                t=tuple(l + [z])
            else:
                t=tuple(l+list(z.args))
            return eval(str(x.func) + str(t))
    
    return x.subs(y,z)

In [28]:
def tseitin1(s):
    # for binary operation
    
    expr = get_sp_clause(s)
    s=symbol_init("p")
    if expr.func == Symbol:
        return true
    else:
        return tseitin_aux(sorted(subformulas(expr), key=depth), s) 

def tseitin_aux(L, s):
    #nonlocal newvar
    newvar = s()
    if (L==[]):
        return true
    if depth(L[0]) > 0:
        if len(L[1:])>0 :
            #newList = [x.subs(L[0], newvar) for x in L[1:]]  
            
            newList = [binSubs(x, L[0], newvar) for x in L[1:]]  
            
            return Equivalent(newvar, L[0]) & tseitin_aux(newList, s)
        else:      
            return Equivalent(newvar, L[0]) & newvar
    else:
        print("error")

In [35]:
sp.to_cnf(tseitin1("1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))"))

p9 & (p1 | x1) & (p2 | x2) & (p3 | x5) & (p1 | ~p5) & (p2 | ~p4) & (p3 | ~p4) & (p4 | ~p5) & (p5 | ~p6) & (p6 | ~p7) & (p7 | ~p8) & (p8 | ~p9) & (x1 | ~p9) & (x2 | ~p8) & (x3 | ~p7) & (x4 | ~p6) & (~p1 | ~x1) & (~p2 | ~x2) & (~p3 | ~x5) & (p4 | ~p2 | ~p3) & (p5 | ~p1 | ~p4) & (p6 | ~p5 | ~x4) & (p7 | ~p6 | ~x3) & (p8 | ~p7 | ~x2) & (p9 | ~p8 | ~x1)

In [36]:
# given(tseitin1)